In [11]:
from dotenv import load_dotenv
from pathlib import Path
import os
import mysql.connector
import pandas as pd

env_path = "/Users/rang/Desktop/PJ/PY/env.env"
load_dotenv(dotenv_path=env_path)

def get_connection():
    conn = mysql.connector.connect(
        host=os.getenv("db_host"),
        database=os.getenv("db_database"),
        user=os.getenv("db_username"),
        password=os.getenv("db_password")
    )
    return conn

def is_invalid(value):
    return pd.isna(value) or value == ""

# ####################################### #
# get_file_lst
# 파일 경로와 목록 정보를 조회
# ####################################### #
def get_file_lst(assess_type, question_cd, question_no=None, order_num=None):
    conn = None
    cursor = None
    try:
        conn = get_connection()
        cursor = conn.cursor()

        sql = ""
        sql +=  "select lst.ORDER_NUM, lst.ASSESS_TYPE, lst.QUESTION_CD, lst.QUESTION_NO, lst.QUESTION_MINOR_NO "
        sql +=  "   , concat(lst.MAIN_PATH,'/',lst.SUB_PATH) as PATH, lst.FILE_NAME, lst.DURATION, lst.RATE "
        sql +=  "   , ref.SCORE, alc.SCORE_ALLOCATION as alc_score, alc.note "
        sql +=  "from assess_file_lst lst "
        sql +=  "   inner join assess_lst alst "
        sql +=  "      on lst.PATIENT_ID = alst.PATIENT_ID "
        sql +=  "      and lst.ORDER_NUM = alst.ORDER_NUM "
        sql +=  "      and alst.EXCLUDED = '0' "
        sql +=  "   inner join assess_score_reference ref "
        sql +=  "      on lst.PATIENT_ID = ref.PATIENT_ID "
        sql +=  "      and lst.ORDER_NUM = ref.ORDER_NUM "
        sql +=  "      and lst.ASSESS_TYPE = ref.ASSESS_TYPE "
        sql +=  "      and lst.QUESTION_CD = ref.QUESTION_CD "
        sql +=  "      and lst.QUESTION_NO = ref.QUESTION_NO "
        sql +=  "       and ref.USE_YN = 'Y' "
        sql +=  "   left outer join assess_score_allocation alc "
        sql +=  "      on lst.ASSESS_TYPE = alc.ASSESS_TYPE "
        sql +=  "      and lst.QUESTION_CD = alc.QUESTION_CD "
        sql +=  "      and lst.QUESTION_NO = alc.QUESTION_NO "
        sql += f"where lst.ASSESS_TYPE = '{assess_type}' "
        sql += f"and lst.QUESTION_CD = '{question_cd}' "
        if question_no:
            sql += f"and lst.QUESTION_NO = {question_no} "
        if order_num:
            sql += f"and lst.ORDER_NUM = {order_num} "
        sql +=  "and lst.USE_YN = 'Y' "
    
        cursor.execute(sql)
        rows = cursor.fetchall()
        ret_df = pd.DataFrame(rows, columns=['ORDER_NUM', 'ASSESS_TYPE', 'QUESTION_CD', 'QUESTION_NO', 'QUESTION_MINOR_NO', 'Path','File Name','Duration', 'Rate', 'Score(Refer)', 'Score(Alloc)', 'Note' ])

        msg = f'{len(ret_df)}건의 데이터가 조회되었습니다.'
        return msg, ret_df

    except Exception as e:
        return f"오류 발생: {str(e)}", None

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [45]:
_, temp = get_file_lst("CLAP_A", "SAY_OBJ")
clapdata = temp[["QUESTION_NO","Path", "File Name", "Score(Refer)"]]
clapdata["Patient_ID"] = clapdata["Path"].str.split("/").str[0]
clapdata["Score(Refer)"] = pd.to_numeric(clapdata["Score(Refer)"], errors="coerce")
clapdata["Total_Score"] = clapdata.groupby("Patient_ID")["Score(Refer)"].transform("sum").astype(float)
clapdata

/var/folders/72/bfx4dl3x59z_2lk5805zqw900000gn/T/ipykernel_51544/3448570349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clapdata["Patient_ID"] = clapdata["Path"].str.split("/").str[0]
/var/folders/72/bfx4dl3x59z_2lk5805zqw900000gn/T/ipykernel_51544/3448570349.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clapdata["Score(Refer)"] = pd.to_numeric(clapdata["Score(Refer)"], errors="coerce")
/var/folders/72/bfx4dl3x59z_2lk5805zqw900000gn/T/ipykernel_51544/3448570349.py:5: SettingWithCopyWarning: 
A

,QUESTION_NO,Path,File Name,Score(Refer),Patient_ID,Total_Score
0,1,1001/clap_a/5,p_1_0.wav,2.0,1001,20.0
1,2,1001/clap_a/5,p_2_0.wav,2.0,1001,20.0
2,3,1001/clap_a/5,p_3_0.wav,2.0,1001,20.0
3,4,1001/clap_a/5,p_4_0.wav,2.0,1001,20.0
4,5,1001/clap_a/5,p_5_0.wav,2.0,1001,20.0
...,...,...,...,...,...,...
595,6,5067/clap_a/5,p_6_0.wav,2.0,5067,16.0
596,7,5067/clap_a/5,p_7_0.wav,2.0,5067,16.0
597,8,5067/clap_a/5,p_8_0.wav,2.0,5067,16.0
598,9,5067/clap_a/5,p_9_0.wav,0.0,5067,16.0


In [46]:
patients = clapdata["Patient_ID"].unique()
patients

array(['1001', '1003', '1004', '1015', '1023', '1024', '1025', '1026',
       '1027', '1032', '1033', '1034', '1035', '1036', '1042', '1043',
       '1045', '2006', '2007', '2014', '3008', '3009', '3010', '3011',
       '3017', '3018', '3019', '3020', '3022', '3028', '3029', '3030',
       '3037', '3038', '3039', '3040', '3041', '3070', '3071', '3074',
       '3075', '3076', '4046', '4047', '4052', '4053', '4054', '4055',
       '4056', '4057', '5058', '5059', '5060', '5061', '5062', '5063',
       '5064', '5065', '5066', '5067'], dtype=object)

In [47]:
order = [f"p_{i}_0.wav" for i in range(1, 11)]

variances = (
    clapdata[clapdata["File Name"].isin(order)]
      .groupby("File Name")["Score(Refer)"]
      .var()
)

variances_sorted = variances.sort_values(ascending=False)
print(variances_sorted)
# 무지개(6) -> 그네(9) -> 해바라기(3) -> 태극기(4) ,,,

File Name
p_6_0.wav     0.952260
p_9_0.wav     0.898023
p_3_0.wav     0.825989
p_4_0.wav     0.762429
p_2_0.wav     0.728814
p_10_0.wav    0.693785
p_8_0.wav     0.693785
p_7_0.wav     0.659887
p_5_0.wav     0.620056
p_1_0.wav     0.525141
Name: Score(Refer), dtype: float64


In [48]:
score_p6 = (
    clapdata[clapdata["File Name"] == "p_6_0.wav"]
    .set_index("Patient_ID")["Score(Refer)"]
)

score_p9 = (
    clapdata[clapdata["File Name"] == "p_9_0.wav"]
    .set_index("Patient_ID")["Score(Refer)"]
)

total_scores = (
    clapdata.drop_duplicates("Patient_ID")
    .set_index("Patient_ID")["Total_Score"]
)

df_corr = pd.concat([score_p6, score_p9, total_scores], axis=1).dropna()
df_corr.columns = ["Score_p6", "Score_p9", "Total_Score"]

corr_value_06 = df_corr["Score_p6"].corr(df_corr["Total_Score"])
corr_value_09 = df_corr["Score_p9"].corr(df_corr["Total_Score"])

print("무지개 & total 점수 간 상관계수:", corr_value_06)
print("그네 & total 점수 간 상관계수:", corr_value_09)

무지개 & total 점수 간 상관계수: 0.8207937738544735
그네 & total 점수 간 상관계수: 0.7883531553200066


In [49]:
people = []
base_dir = "/Users/rang/Documents/Clapdata"

for patient_id in patients:
    row = clapdata.loc[clapdata["Patient_ID"] == patient_id, "Total_Score"]
    total_score = row.iloc[0]

    rnbw_path, swng_path = None, None

    for prompt_id in [9, 6]:      # 9: 무지개, 6: 그네 (분산 1,2등)
        audio = f"{base_dir}/{patient_id}/CLAP_A/5/p_{prompt_id}_0.wav"

        if prompt_id == 9:
            rnbw_path = audio
        elif prompt_id == 6:
            swng_path = audio

    if rnbw_path and swng_path:
          people.append({
              'pid': patient_id,
              'rainbow': rnbw_path,
              'swing': swng_path,
              'total_score': total_score
          })

people

[{'pid': '1001',
  'rainbow': '/Users/rang/Documents/Clapdata/1001/CLAP_A/5/p_9_0.wav',
  'swing': '/Users/rang/Documents/Clapdata/1001/CLAP_A/5/p_6_0.wav',
  'total_score': np.float64(20.0)},
 {'pid': '1003',
  'rainbow': '/Users/rang/Documents/Clapdata/1003/CLAP_A/5/p_9_0.wav',
  'swing': '/Users/rang/Documents/Clapdata/1003/CLAP_A/5/p_6_0.wav',
  'total_score': np.float64(20.0)},
 {'pid': '1004',
  'rainbow': '/Users/rang/Documents/Clapdata/1004/CLAP_A/5/p_9_0.wav',
  'swing': '/Users/rang/Documents/Clapdata/1004/CLAP_A/5/p_6_0.wav',
  'total_score': np.float64(20.0)},
 {'pid': '1015',
  'rainbow': '/Users/rang/Documents/Clapdata/1015/CLAP_A/5/p_9_0.wav',
  'swing': '/Users/rang/Documents/Clapdata/1015/CLAP_A/5/p_6_0.wav',
  'total_score': np.float64(20.0)},
 {'pid': '1023',
  'rainbow': '/Users/rang/Documents/Clapdata/1023/CLAP_A/5/p_9_0.wav',
  'swing': '/Users/rang/Documents/Clapdata/1023/CLAP_A/5/p_6_0.wav',
  'total_score': np.float64(14.0)},
 {'pid': '1024',
  'rainbow': '/Use

In [50]:
# ========= 라이브러리 Import ==========
import os  # 파일 경로를 다룰 때 사용
import numpy as np  # 숫자 배열 처리용
import librosa  # 오디오 로딩 및 특징 추출용 라이브러리
import tensorflow as tf  # 딥러닝 프레임워크
from sklearn.model_selection import train_test_split
from transformers import WhisperProcessor, WhisperForConditionalGeneration  # Huggingface Whisper Tokenizer & Model
import torch  # PyTorch (Whisper는 torch 기반)

# ========== 하이퍼파라미터 설정 ==========
SAMPLE_RATE = 16000  # 오디오 샘플링 레이트
N_MELS = 128  # Mel-Spectrogram의 mel band 수
MAX_TOKEN_LENGTH = 512  # Whisper 토큰 시퀀스 길이 (패딩 포함)
TOKEN_EMBED_DIM = 32  # Token 임베딩 차원 수
LSTM_UNITS = 32  # LSTM hidden unit 개수
DROPOUT = 0.3  # Dropout 비율
LR = 1e-4  # Learning rate=
EPOCHS = 100  # 학습 Epoch 수
BATCH_SIZE = 2  # Batch size
TEMPERATURE = 0  # Whisper generate temperature (0 = deterministic)

# ========== Whisper 모델 초기화 ==========
device = torch.device("mps" if torch.backends.mps.is_available()                # macOS
                      else ("cuda" if torch.cuda.is_available() else "cpu"))    # windos # GPU 사용 가능하면 cuda, 아니면 cpu 사용
processor = WhisperProcessor.from_pretrained("openai/whisper-small")            # Whisper Tokenizer 불러오기
whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device) # Whisper 모델 불러오기 (디바이스 할당)
forced_ids = processor.get_decoder_prompt_ids(language="ko", task="transcribe")

# 토크나이저/패딩
_vocab_ids = list(processor.tokenizer.get_vocab().values())
BASE_VOCAB = max(_vocab_ids) + 1    # 추가 토큰까지 포함한 전체 vocab의 "최대 토큰ID + 1"

# ========== 오디오 파일을 Mel-Spectrogram으로 변환 ==========
def wav_to_mel(wav_path, sr=SAMPLE_RATE, n_mels=N_MELS):
    y, _ = librosa.load(wav_path, sr=sr)                                # 오디오 파일 로드 (sr로 리샘플링)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)     # Mel-Spectrogram 추출 (128개 멜 필터 사용)
    mel_db = librosa.power_to_db(mel, ref=np.max)                       # dB 단위로 변환 (정규화 효과)
    mel_db = mel_db[..., np.newaxis]                                    # 차원 추가 → (n_mels, T, 1)
    return mel_db.astype(np.float32)                                    # float32로 캐스팅해서 반환

# ========== 오디오 파일을 Whisper Token ID로 변환 ==========
def wav_to_token_ids(wav_path, sr=SAMPLE_RATE):
    y, _ = librosa.load(wav_path, sr=sr)                                # 오디오 로드
    inputs = processor(y, sampling_rate=sr, return_tensors="pt", return_attention_mask=True) # Whisper Processor로 전처리
    input_features = inputs.input_features.to(device)                   # Whisper 입력 포맷 (Device 할당)
    # attention_mask = inputs.attention_mask.to(device)
    pred_ids = whisper_model.generate(                                  # Whisper로 Token ID 예측                             
        input_features,
        # attention_mask=attention_mask,
        forced_decoder_ids=forced_ids,
        temperature=TEMPERATURE) 
     
    token_ids = pred_ids[0].cpu().tolist()                              # 결과를 CPU로 옮기고 list로 변환
    if len(token_ids) < MAX_TOKEN_LENGTH:                               # 토큰이 짧은 경우 패딩
        token_ids += [0] * (MAX_TOKEN_LENGTH - len(token_ids))
    else:
        token_ids = token_ids[:MAX_TOKEN_LENGTH]                        # 토큰이 긴 경우 MAX_TOKEN_LENGTH 길이로 잘라내기 
    return np.array(token_ids, dtype=np.int32)                          # numpy array로 반환

# ========== Mel-Spectrogram 길이 맞추기 (Padding) ==========
def pad_mels(mel_list):
    max_time = max(m.shape[1] for m in mel_list)                                # 가장 긴 time 길이 계산
    batch = np.zeros((len(mel_list), N_MELS, max_time, 1), dtype=np.float32)    # Zero-padding 적용하여 mel_batch 초기화 (batch, 128, max_time, 1)
    for i, mel in enumerate(mel_list):
        batch[i, :, :mel.shape[1], :] = mel                                     # 데이터가 있는 부분만 복사, 나머지는 0
    return batch

# ========== 데이터를 담을 리스트 초기화 ==========
people = []
base_dir = "/Users/rang/Documents/Clapdata"

for patient_id in patients:
    row = clapdata.loc[clapdata["Patient_ID"] == patient_id, "Total_Score"]
    total_score = row.iloc[0]

    rnbw_path, swng_path = None, None

    for prompt_id in [9, 6]:      # 9: 무지개, 6: 그네 (분산 1,2등)
        audio = f"{base_dir}/{patient_id}/CLAP_A/5/p_{prompt_id}_0.wav"

        if prompt_id == 9:
            rnbw_path = audio
        elif prompt_id == 6:
            swng_path = audio

    if rnbw_path and swng_path:
          people.append({
              'pid': patient_id,
              'rainbow': rnbw_path,
              'swing': swng_path,
              'total_score': total_score
          })

# 각 데이터에 대해 Mel-Spectrogram과 Token ID를 추출하여 리스트에 저장
mel_r_list, mel_s_list, tok_r_list, tok_s_list, y_list = [], [], [], [], []

from tqdm import tqdm

for person in tqdm(people):   
    mel_r = wav_to_mel(person['rainbow'])
    mel_s = wav_to_mel(person['swing'])
    tok_r = wav_to_token_ids(person['rainbow'])
    tok_s = wav_to_token_ids(person['swing'])

    mel_r_list.append(mel_r)
    mel_s_list.append(mel_s)
    tok_r_list.append(tok_r)
    tok_s_list.append(tok_s)

    # 총점은 0~1로 정규화
    y_list.append(person['total_score'] / 20)

mel_r_data = pad_mels(mel_r_list)
mel_s_data = pad_mels(mel_s_list)
tok_r_data = np.stack(tok_r_list).astype(np.int32)
tok_s_data = np.stack(tok_s_list).astype(np.int32)
y_data = np.array(y_list, dtype=np.float32).reshape(-1, 1)

100%|██████████| 60/60 [00:55<00:00,  1.08it/s]


In [51]:
# =========================
# Model
# ========================= 
# 사용자 정의 인코더
L = tf.keras.layers

def build_encoder(name_prefix="enc"):
    mel_input = L.Input(shape=(N_MELS, None, 1), name=f'{name_prefix}_mel_in')
    tok_input = L.Input(shape=(MAX_TOKEN_LENGTH,), name=f'{name_prefix}_tok_in')

    # Mel branch
    x1 = L.Permute((2, 1, 3))(mel_input)                    # (time, mel, 1)
    x1 = L.Reshape((-1, N_MELS))(x1)                        # (time, mel)
    x1 = L.Masking(mask_value=0.0)(x1)
    x1 = L.Bidirectional(L.LSTM(LSTM_UNITS, return_sequences=True))(x1)
    x1 = L.Dropout(DROPOUT)(x1)

    # Token branch
    x2 = L.Embedding(input_dim=BASE_VOCAB, output_dim=TOKEN_EMBED_DIM, mask_zero=True)(tok_input)
    x2 = L.Bidirectional(L.LSTM(LSTM_UNITS, return_sequences=True))(x2)
    x2 = L.Dropout(DROPOUT)(x2)

    # Cross Attention (mel queries token)
    x = L.MultiHeadAttention(num_heads=4, key_dim=64)(query=x1, value=x2, key=x2)
    x = L.Bidirectional(L.LSTM(LSTM_UNITS, return_sequences=False))(x)
    x = L.Dropout(DROPOUT)(x)            
    emb = L.Dense(128, activation='relu')(x)
    return tf.keras.Model([mel_input, tok_input], emb, name=f'{name_prefix}_encoder')

encoder = build_encoder("shared")

In [52]:
# ----- 무지개/그네 입력 -----
mel_r = L.Input(shape=(N_MELS, None, 1), name='mel_rainbow')
tok_r = L.Input(shape=(MAX_TOKEN_LENGTH,), name='tok_rainbow')
mel_s = L.Input(shape=(N_MELS, None, 1), name='mel_swing')
tok_s = L.Input(shape=(MAX_TOKEN_LENGTH,), name='tok_swing')

emb_r = encoder([mel_r, tok_r])  # 사용자 정의 인코더
emb_s = encoder([mel_s, tok_s])

combined = L.Concatenate()([emb_r, emb_s])

x = L.Dense(256, activation='relu')(combined)
x = L.Dropout(0.4)(x)
x = L.Dense(128, activation='relu')(x)
x = L.Dropout(0.3)(x)
out = L.Dense(1, activation='sigmoid', name='total_over_20')(x)  # 총점/20

model = tf.keras.Model(inputs=[mel_r, tok_r, mel_s, tok_s], outputs=out, name="SayObjTotalRegressor")

model.compile(optimizer=tf.keras.optimizers.Adam(LR), loss='mse', metrics=['mae'])
model.summary()

Model: "SayObjTotalRegressor"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ mel_rainbow         │ (None, 128, None, │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tok_rainbow         │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mel_swing           │ (None, 128, None, │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tok_swing           │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_encoder      │ (None, 128)       │  1,817,056 │ mel_rainbow[0][0… │
│ (Functional)        │                   │            │ tok_rainbow[0][0… │
│                     │                   │            │ mel_swing[0][0],  │
│                     │                   │            │ tok_swing[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256)       │          0 │ shared_encoder[0… │
│ (Concatenate)       │                   │            │ shared_encoder[1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │     65,792 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 256)       │          0 │ dense_4[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     32,896 │ dropout_11[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 128)       │          0 │ dense_5[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ total_over_20       │ (None, 1)         │        129 │ dropout_12[0][0]  │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,915,873 (7.31 MB)

 Trainable params: 1,915,873 (7.31 MB)

 Non-trainable params: 0 (0.00 B)

In [53]:
# ========== train, test 데이터 나누기 ==========
y_flat = y_data.ravel()
y_bins = np.digitize(y_flat, bins=[0.33, 0.66]) # 점수를 기준으로 층화 분할 하기 위해 구간 나눠서 범주 만들기

# Mel, Token, Label을 train/test로 나누기
mel_r_train, mel_r_test, mel_s_train, mel_s_test, token_r_train, token_r_test, token_s_train, token_s_test, y_train, y_test = train_test_split(
    mel_r_data, mel_s_data, tok_r_data, tok_s_data, y_data, test_size=0.3, random_state=3, stratify=y_bins
)

In [54]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(filepath="say_obj_model.keras", monitor='val_loss', save_best_only=True)
]

history = model.fit(
    x={'mel_rainbow': mel_r_train, 'tok_rainbow': token_r_train,
     'mel_swing': mel_s_train, 'tok_swing': token_s_train},
    y=y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(
        {'mel_rainbow': mel_r_test, 'tok_rainbow': token_r_test,
        'mel_swing': mel_s_test, 'tok_swing': token_s_test}, y_test),
    callbacks=callbacks,
    verbose=1
)

Epoch 1/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 10s 245ms/step - loss: 0.1798 - mae: 0.4002 - val_loss: 0.1944 - val_mae: 0.4143
Epoch 2/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 214ms/step - loss: 0.1769 - mae: 0.3972 - val_loss: 0.1894 - val_mae: 0.4095
Epoch 3/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 215ms/step - loss: 0.1688 - mae: 0.3865 - val_loss: 0.1755 - val_mae: 0.3950
Epoch 4/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 212ms/step - loss: 0.1588 - mae: 0.3718 - val_loss: 0.1488 - val_mae: 0.3565
Epoch 5/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 253ms/step - loss: 0.1368 - mae: 0.3319 - val_loss: 0.1396 - val_mae: 0.3350
Epoch 6/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 277ms/step - loss: 0.1256 - mae: 0.3060 - val_loss: 0.1329 - val_mae: 0.3034
Epoch 7/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 282ms/step - loss: 0.1255 - mae: 0.2913 - val_loss: 0.1301 - val_mae: 0.3085
Epoch 8/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 277ms/step - loss: 0.1147 - mae: 0.2758 - val_loss: 0.1256 - val_mae: 0.2894
Epoch 9/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 280ms

In [65]:
model = tf.keras.models.load_model("say_obj_model.keras")

In [64]:
model.evaluate([mel_r_test, token_r_test, mel_s_test, token_s_test], y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step - loss: 0.0579 - mae: 0.1534


[0.05789696425199509, 0.1533646136522293]

In [57]:
y_pred_norm = model.predict(
    {'mel_rainbow': mel_r_data, 'tok_rainbow': tok_r_data,
     'mel_swing':   mel_s_data, 'tok_swing':   tok_s_data},
    verbose=1
)

y_pred_flatten = y_pred_norm.flatten()

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step  


In [58]:
y_pred_flatten

array([0.9357723 , 0.9879068 , 0.9879051 , 0.987905  , 0.7436098 ,
       0.88531643, 0.96147835, 0.94657177, 0.96518636, 0.9772789 ,
       0.98790807, 0.7468336 , 0.9732477 , 0.9286314 , 0.99359816,
       0.9619211 , 0.98517627, 0.99359816, 0.993598  , 0.9571279 ,
       0.9680128 , 0.8553059 , 0.8631245 , 0.8694044 , 0.66061896,
       0.43473276, 0.91055346, 0.41302037, 0.38739252, 0.62755144,
       0.69936186, 0.44303927, 0.34545732, 0.94426   , 0.9703165 ,
       0.807349  , 0.98790807, 0.86766523, 0.30154333, 0.9806469 ,
       0.48935476, 0.89853823, 0.49630225, 0.36767152, 0.9463269 ,
       0.71214044, 0.3805746 , 0.9901805 , 0.4824956 , 0.46807027,
       0.3529467 , 0.8826527 , 0.9899569 , 0.9633293 , 0.7637886 ,
       0.82708186, 0.56615216, 0.91523176, 0.37599513, 0.7474683 ],
      dtype=float32)

In [59]:
y_pred_flatten * 20

array([18.715446 , 19.758137 , 19.758102 , 19.7581   , 14.872196 ,
       17.70633  , 19.229567 , 18.931435 , 19.303726 , 19.545578 ,
       19.758162 , 14.936672 , 19.464954 , 18.572628 , 19.871964 ,
       19.238422 , 19.703526 , 19.871964 , 19.87196  , 19.14256  ,
       19.360256 , 17.106117 , 17.26249  , 17.388088 , 13.212379 ,
        8.694655 , 18.21107  ,  8.260407 ,  7.7478504, 12.551029 ,
       13.987237 ,  8.8607855,  6.9091463, 18.8852   , 19.40633  ,
       16.14698  , 19.758162 , 17.353304 ,  6.0308666, 19.612938 ,
        9.787095 , 17.970764 ,  9.926044 ,  7.3534303, 18.926538 ,
       14.242809 ,  7.611492 , 19.80361  ,  9.649912 ,  9.361405 ,
        7.058934 , 17.653053 , 19.799139 , 19.266586 , 15.275772 ,
       16.541637 , 11.323043 , 18.304636 ,  7.5199027, 14.949366 ],
      dtype=float32)

In [60]:
np.round(y_pred_flatten * 20)

array([19., 20., 20., 20., 15., 18., 19., 19., 19., 20., 20., 15., 19.,
       19., 20., 19., 20., 20., 20., 19., 19., 17., 17., 17., 13.,  9.,
       18.,  8.,  8., 13., 14.,  9.,  7., 19., 19., 16., 20., 17.,  6.,
       20., 10., 18., 10.,  7., 19., 14.,  8., 20., 10.,  9.,  7., 18.,
       20., 19., 15., 17., 11., 18.,  8., 15.], dtype=float32)

In [61]:
y_pred = np.round(y_pred_flatten * 20)
y_true = y_data.flatten() * 20

pearson_r = np.corrcoef(y_pred, y_true)[0, 1]
print(f"피어슨 상관계수: {pearson_r:.4f}")

피어슨 상관계수: 0.9028


In [62]:
say_obj = pd.DataFrame({
    "환자 ID": [p['pid'] for p in people],
    "예측 총점": y_pred,
    "실제 총점": y_data.flatten() * 20
})

print(say_obj.head())

say_obj.to_excel("say_object_result.xlsx", index=False)
print("예측 결과 저장 완료: say_object_result.xlsx")

  환자 ID  예측 총점  실제 총점
0  1001   19.0   20.0
1  1003   20.0   20.0
2  1004   20.0   20.0
3  1015   20.0   20.0
4  1023   15.0   14.0
예측 결과 저장 완료: say_object_result.xlsx
